# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#create dataframe of city weather from csv
city_weather_df = pd.read_csv("../output/citiesweather.csv", index_col = 0)

#preview frame
city_weather_df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Alekseyevsk,57.8406,108.3492,54.72,51,100,3.74,RU,1623600451
1,Bethel,41.3712,-73.4140,79.48,64,40,6.91,US,1623600451
2,Nuevo Progreso,23.1667,-97.9500,86.88,56,75,6.51,MX,1623600452
3,Camacha,33.0833,-16.3333,71.53,73,75,16.11,PT,1623600452
4,Atuona,-9.8000,-139.0333,79.12,78,49,18.19,PF,1623600191


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=gkey)

#customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

#define locations of interest and level of humidity
locations = city_weather_df[["Latitude", "Longitude"]]
humid = city_weather_df["Humidity"]

#build heatlayer
humidity_layer = gmaps.heatmap_layer(locations, weights = humid, 
                                 dissipating = True, max_intensity = 100,
                                 point_radius = 15)

#add heatlayer layer to map
fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#create dataframe of cities that fit weather criteria
ideal_weather_df = city_weather_df.loc[(city_weather_df['Max Temperature'] > 70) & (city_weather_df['Max Temperature'] < 80) & (city_weather_df['Wind Speed'] < 10) & (city_weather_df['Cloudiness'] == 0),  :]

#view frame
ideal_weather_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
43,Ziarat,30.3823,67.7282,73.49,8,0,9.78,PK,1623600463
56,Oruzgan,32.8333,66.0000,78.58,9,0,6.69,AF,1623600532
93,Bataiporã,-22.2953,-53.2711,73.33,45,0,6.82,BR,1623600607
189,Fukuechō,32.6881,128.8419,70.77,89,0,3.47,JP,1623600766
253,Itirapina,-22.2528,-47.8228,70.52,44,0,6.85,BR,1623600849
259,Calama,-22.4667,-68.9333,70.29,4,0,9.22,CL,1623600851
260,Saint-Pierre,-21.3393,55.4781,73.04,79,0,6.91,RE,1623600851
361,Catanduva,-21.1378,-48.9728,75.49,26,0,7.54,BR,1623601010
409,Bela Vista,-22.1089,-56.5211,78.51,30,0,5.70,BR,1623601155
438,Schwetzingen,49.3781,8.5819,78.21,34,0,9.22,DE,1623601163


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#store into new dataframe named hotel_df
hotel_df = ideal_weather_df

#create hotel name column within dataframe
hotel_df['Hotel Name'] = ''

#set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = { 
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "key": gkey,
}
#iterate through pandas dataframe
for index, row in hotel_df.iterrows():

#define city and location
    lat = row['Latitude']
    lng = row['Longitude']
    city = row['City']
    
#set location variable within call specifications
    target_coordinates = f'{lat}, {lng}'

#add location parameter
    params['location'] = target_coordinates

    print(f"Retrieving Results for {city}.")

#assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
#extract results
    results = response['results']
    
#add found hotel to dataframe
    try:
        print(f"Closest hotel to {city} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
#add 'none' to dataframe, if no hotel found or error
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "None"

        print("Missing field/result or no hotel within range.")
        
    print("------------")

ipykernel_launcher:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Retrieving Results for Ziarat.
Closest hotel to Ziarat is Grand Hotel.
------------
Retrieving Results for Oruzgan.
C:\Users\12164\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
Missing field/result or no hotel within range.
------------
Retrieving Results for Bataiporã.
Closest hotel to Bataiporã is Hotel Vale do Ivinhema.
------------
Retrieving Resul

In [6]:
#create new dataframe of only cities with hotels withing 5000m of coordinates
hotel_clean_df = hotel_df.loc[hotel_df['Hotel Name'] != 'None', :]
hotel_clean_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
43,Ziarat,30.3823,67.7282,73.49,8,0,9.78,PK,1623600463,Grand Hotel
93,Bataiporã,-22.2953,-53.2711,73.33,45,0,6.82,BR,1623600607,Hotel Vale do Ivinhema
189,Fukuechō,32.6881,128.8419,70.77,89,0,3.47,JP,1623600766,Hotel Sou
253,Itirapina,-22.2528,-47.8228,70.52,44,0,6.85,BR,1623600849,Pousada Paraiso das Aguas
259,Calama,-22.4667,-68.9333,70.29,4,0,9.22,CL,1623600851,Geotel Calama
260,Saint-Pierre,-21.3393,55.4781,73.04,79,0,6.91,RE,1623600851,Le Battant Des Lames
361,Catanduva,-21.1378,-48.9728,75.49,26,0,7.54,BR,1623601010,Paradise Inn Hotel
409,Bela Vista,-22.1089,-56.5211,78.51,30,0,5.70,BR,1623601155,Hotel Pousada da Fronteira
438,Schwetzingen,49.3781,8.5819,78.21,34,0,9.22,DE,1623601163,See Hotel & Restaurant Die Ente
479,Vallenar,-28.5708,-70.7581,70.14,34,0,3.40,CL,1623601240,Hotel Puerto de Vega


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_clean_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]
locations = hotel_clean_df[["Latitude", "Longitude"]]

In [8]:
#build marker layer
markers = gmaps.marker_layer(locations, info_box_content= hotel_clean_info)

#add layer to figure with heatmap
fig.add_layer(markers)

#display
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…